In [1]:
import datetime

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense

# Imports for the HParams plugin
from tensorboard.plugins.hparams import api_pb2
from tensorboard.plugins.hparams import summary as hparams_summary
from google.protobuf import struct_pb2

In [3]:
# Get MNIST dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0

# Convolutional Neural Network

Network architect (from input to output):
* Conv2D
    * Number of kernels: 32
    * Kernel size: hyperparamters
* Conv2D
    * Number of kernels: 64
    * Kernel size: hyperparamters
* Maxpooling
    * Kernel size: (2, 2)
* Dropout
* Dense
    * Number of units: hyperparameters
* Dropout
* Dense
    * Number of units: 10

With the overal architect being kept constant, the following hyperparameters are tuned:
1. Learning rate
2. Size of kernel used in Conv2D layers (2 Conv2D layers use kernels of the same size)
3. Dropout rate
4. Number of units on the first Dense layer


In [5]:
# Set up hyperparameters lists
learning_rate_list = [1e-3, 1e-2, 1e-1]
kernel_size_list = [3, 5]
dropout_rate_list = [0, 0.25, 0.5]
dense_units_list = [32, 128]

In [ ]:
# Add hyperparamters to Hparam 
def create_experiment_summary(learning_rate_list, dropout_rate_list, kernel_size_list, dense_units_list):
    learning_rate_list_val = struct_pb2.ListValue()
    learning_rate_list_val.extend(learning_rate_list)
    
    dropout_rate_list_val = struct_pb2.ListValue()
    dropout_rate_list_val.extend(dropout_rate_list)
    
    kernel_size_list_val = struct_pb2.ListValue()
    kernel_size_list_val.extend(kernel_size_list)
    
    dense_units_list_val = struct_pb2.ListValue()
    dense_units_list_val.extend(dense_units_list)
    
    return hparams_summary.experiment_pb(
      # The hyperparameters being changed
      hparam_infos=[
          api_pb2.HParamInfo(name='learning_rate',
                             display_name='Learning Rate',
                             type=api_pb2.DATA_TYPE_FLOAT64,
                             domain_discrete=learning_rate_list_val),
          api_pb2.HParamInfo(name='kernel_size',
                             display_name='Kernel Size',
                             type=api_pb2.DATA_TYPE_FLOAT64,
                             domain_discrete=kernel_size_list_val),
          api_pb2.HParamInfo(name='dropout_rate',
                             display_name='Dropout rate',
                             type=api_pb2.DATA_TYPE_FLOAT64,
                             domain_discrete=dropout_rate_list_val),
          api_pb2.HParamInfo(name='dense_units',
                             display_name='Dense Units',
                             type=api_pb2.DATA_TYPE_FLOAT64,
                             domain_discrete=dense_units_list_val)
      ],
      # The metrics being tracked
      metric_infos=[
          api_pb2.MetricInfo(
              name=api_pb2.MetricName(
                  tag='accuracy'),
              display_name='Accuracy'),
      ]
    )

exp_summary = create_experiment_summary(learning_rate_list, dropout_rate_list, kernel_size_list, dense_units_list)
root_logdir_writer = tf.summary.create_file_writer("logs_cnn/hparam_tuning")
with root_logdir_writer.as_default():
    tf.summary.import_event(tf.compat.v1.Event(summary=exp_summary).SerializeToString())